In [1]:
# importing modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from modules.library import documentRetrieval as dr
from modules.library import queryExp as qe
from modules.library.postgresql import PostgresQL
pg = PostgresQL() 


C:\Users\sarab\Miniconda3\envs\envirolens\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [15]:
# importing word vectors
from gensim.models import KeyedVectors
wiki_en_align = './../data/fasttext/wiki.en.align.vec' #'../../data/fasttext/wiki.en.align.vec'
# get fasttext wiki embeddings for english
wv_wiki_en = KeyedVectors.load_word2vec_format(wiki_en_align)
print('english words {}'.format(len(list(wv_wiki_en.vocab.keys()))))

C:\Users\sarab\Miniconda3\envs\envirolens\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


english words 2519370


In [16]:
# prepare stopword list
from nltk.corpus   import stopwords
from nltk.tokenize import word_tokenize
stopWords = stopwords.words('english')

In [2]:

pg.connect(database="envirolens", user="postgres", password="dbpass") #"eurlex_env_only" "solata.2018"

In [7]:
def db_query(query_words):
    """ Iz baze vrne seznam slovarjev, ki vsebujejo dokumente (tekst) in njihove IDje, ki v tekstu vsebujejo vsaj eno od besed iz seznama(list) query_words"""
    output = '|'.join(query_words)
    SQL = """
            SELECT document_id, fulltext_cleaned FROM documents
            WHERE to_tsvector('english', fulltext_cleaned) @@ to_tsquery('english',""" + '\''+ output + '\'' + """);"""
    documents = pg.execute(SQL)
    return(documents)

In [17]:
tokenized = qe.tokenize('water pollution underground', stopWords)
ext = qe.extend_tokens(tokenized, wv_wiki_en)
candidates = qe.pre_retrieval_KNN('water pollution underground', 5, wv_wiki_en, 10, stopWords,extension=False)

In [11]:
tokenized = ["pollution","fish"]
docs = db_query(tokenized) 
#premisli kaj (tokens, ext, candidates) das v db_query (I guess vse? --> ce dela na osnovnem queryju dovolj dobro pusti osnovni, drugace prevec dokumentov)

In [9]:
print(len(docs))

13771


In [10]:
docs[0]

{'document_id': 1000002956,
 'fulltext_cleaned': '7 12 2018 en official journal european union c 443 9 minutes sitting wednesday 20 june 2018 2018 c 443 04 contents 1 summary report workshop 9 2 report economic social partners 9 3 report youth conference 10 4 statement carl b greenidge vice president minister foreign affairs guyana 10 5 statement ekaterina zaharieva deputy prime minister judicial reform minister foreign affairs bulgaria president office eu council 10 6 question time council 10 7 debate council — ‘catch eye’ 11 8 report 15th regional meeting held nairobi kenya east african region 11 13 april 2018 — report co presidents 11 9 approval minutes afternoon sitting monday 18 june 2018 morning afternoon sittings tuesday 19 june 2018 11 10 vote motions resolution included reports submitted three standing committees 12 11 vote urgent motions resolution 13 12 business 13 13 date place 36th session joint parliamentary assembly 13 annex alphabetical list members joint parliamentary 

In [12]:
#ce vzame cas popravi v metriki
def change_dict_structure(dict_list):
    """Takes list of dicts from db_query and changes to dict with key=id, value = text (used for metrices)."""
    texts = {}
    for dic in dict_list:
        doc_id = dic.get('document_id')
        text = dic.get('fulltext_cleaned')
        texts.update({doc_id: text})
    return texts

In [13]:
texts_dict = change_dict_structure(docs)

In [12]:
#texts_dict.get(2)

In [18]:
multiply0 = dr.probability_score(tokenized,texts_dict, dr.probability_multiply,10)
multiply1 = dr.probability_score(tokenized+ext,texts_dict, dr.probability_multiply,10)
multiply2 = dr.probability_score(tokenized+ext+candidates,texts_dict, dr.probability_multiply,10)
# takes also probability_function probability_sum_weight, but doesnt give final result (used in probability_score_sum_weights)
    # top_expansion are the onesthat are not counted as original query, usually ext ((extension by summation of 2 consecutive words) counts as original query
#     """Assigns score to documents based on probability_function metric.
#     Args:
#         tokens (list): List of tokens (tokenized query). If needed also extension (extension by summation of 2 consecutive words and/or KNN candidates).
#         texts (dict):  Keys represent document ids, values are document text. 
#         probability_function (function): Metric function that calculates document relavance. Functions: probability_multiply, probability_sum. Require only first 4 arguments.
#         n (int): Number of returned tuples, sorted by highest scores.
#         top_expansion (list): List of expanded words. Usually candidates (kNN expansion), then have to be also in tokens.
#         alpha (float): Number between 0 and 1. Weight that emphasizes the difference between original query words and expansions. 
#                        For alpha 0.5 all words have same weights (but not same values!), for alpha 1 expansion words have value 0. 
#                        For alpha -1 values equal to cosine similarity to query words. 
#         wv (Word2VecKeyedVectors): Word embeddings.
#     Returns:
#         document_probability (list): Tuples of document ids and scores that measure document relavance. Returns n tuples with highest score.
#     """


In [20]:
multiply0 

[(1000090950, 8.397281148463226e-09),
 (1000098344, 1.8618390146277465e-09),
 (1000100873, 1.7545818547713225e-09),
 (1000099212, 1.3395930920889448e-09),
 (1000096932, 1.303063190942301e-09),
 (1000098889, 7.887012797822669e-10),
 (1000101427, 5.571635189744278e-10),
 (1000101870, 5.520991560422514e-10),
 (1000053798, 4.489271052526326e-10),
 (1000097972, 4.209055552330924e-10)]

In [ ]:
dr.probability_score_sum_weights(original_tokens, top_expansion, texts,n, alpha, wv)
# """As probability_score only weighted.
#         Args:
#         original_tokens(list): List of strings. Tokenized original query. Usually also extension (extension by summation of 2 consecutive words), but not top_expansion (kNN candidates)
#         top_expansion (list): List of expanded words. Usually candidates (kNN expansion).
#         texts (dict):  Keys represent document ids, values are document text.
#         n (int): Number of returned tuples, sorted by highest scores.
#         alpha (float): Number between 0 and 1. Weight that emphasizes the difference between original query words and expansions. 
#                        For alpha 0.5 all words have same weights (but not same values!), for alpha 1 expansion words have value 0. 
#                        For alpha -1 values equal to cosine similarity to query words. 
#         wv (Word2VecKeyedVectors): Word embeddings.
#     Returns:
#         document_score (list): Tuples of document ids and scores that measure document relavance. Returns n tuples with highest score.
#     """

In [18]:
dr.probability_score_sum_weights(tokenized+ext, candidates, texts_dict,10, 0.6, wv_wiki_en)

[(12744, 0.008068733657078818),
 (92986, 0.00802279701885138),
 (92988, 0.0072),
 (3867, 0.006481481481481481),
 (94467, 0.006474820143884891),
 (62937, 0.006428571428571428),
 (9649, 0.006100478468899521),
 (92987, 0.005979543666404406),
 (95196, 0.005958549222797927),
 (30565, 0.005689437508767008)]

In [ ]:
dr.tfidf_score(tokens, texts, tfidf_function,n, *args)
#     """Assigns score to documents based on tfidf_function metric.
#     Args:
#         tokens (list): List of tokens (tokenized query). If needed also extension (extension by summation of 2 consecutive words).For tfidf_sum metric can also add kNN candidates.
#         texts (dict):  Keys represent document ids, values are document text. 
#         probability_function (function): Metric function that calculates document relavance. Functions: tfidf_sum; require only first 4 arguments, tfidf_sum_weight; require all arguments.
#         n (int): Number of returned tuples, sorted by highest scores.
#         top_expansion (list): List of expanded words. Usually candidates (kNN expansion).
#         alpha (float): Number between 0 and 1. Weight that emphasizes the difference between original query words and expansions. 
#                        For alpha 0.5 all words have same weights (but not same values!), for alpha 1 expansion words have value 0. 
#                        For alpha -1 values equal to cosine similarity to query words. 
#         wv (Word2VecKeyedVectors): Word embeddings.
#     Returns:
#         document_probability (list): Tuples of document ids and scores that measure document relavance. Returns n tuples with highest score.
#         not_appear (list): List of words that did not occure in any document.
#     """

In [19]:
dr.tfidf_score(tokenized+ext, texts_dict, dr.tfidf_sum,0)

[(43473, 0.00013343407517938443),
 (43475, 0.00010625765161663372),
 (43481, 6.118553860726017e-05),
 (43501, 0.0010948051614869093),
 (43522, 3.636226322390044e-05),
 (43505, 0.0001343368448742178),
 (43590, 0.0001531879859161154),
 (43591, 6.937542808723318e-05),
 (43496, 1.914090640567437e-05),
 (43523, 0.00034387496988949715),
 (43550, 0.0),
 (43592, 0.00023647653287625772),
 (43528, 0.00010474036884255183),
 (43529, 0.00015994837142101694),
 (43530, 0.0007335801387020908),
 (43551, 0.0008015850228857857),
 (43537, 0.00032578062417419645),
 (43552, 9.18379703037045e-05),
 (43563, 0.00018786410837078976),
 (43556, 6.263638709815322e-05),
 (43567, 1.0476778348131081e-05),
 (43568, 0.00014646306907190016),
 (43542, 0.0001118214548491741),
 (43596, 5.165293275576774e-05),
 (44056, 0.00012821782058015008),
 (43606, 0.000365877580678576),
 (43607, 1.4996095967656829e-05),
 (43569, 0.00017377458758975316),
 (43608, 0.0004127209165920883),
 (43609, 0.000178546758653741),
 (43615, 4.5929737

In [20]:
dr.tfidf_score(tokenized+ext, texts_dict, dr.tfidf_sum_weight,0, candidates, 0.6, wv_wiki_en)

[(43473, 8.006044510763066e-05),
 (43475, 6.375459096998023e-05),
 (43481, 3.67113231643561e-05),
 (43501, 0.0006568830968921455),
 (43522, 2.1817357934340263e-05),
 (43505, 8.248333580880366e-05),
 (43590, 9.638249155658405e-05),
 (43591, 4.16252568523399e-05),
 (43496, 1.1484543843404622e-05),
 (43523, 0.00020632498193369828),
 (43550, 0.0),
 (43592, 0.0001418859197257546),
 (43528, 6.28442213055311e-05),
 (43529, 0.00010430107330797806),
 (43530, 0.00044014808322125445),
 (43551, 0.0004809510137314714),
 (43537, 0.00019546837450451786),
 (43552, 5.5102782182222694e-05),
 (43563, 0.00011271846502247386),
 (43556, 4.040177745304994e-05),
 (43567, 6.286067008878648e-06),
 (43568, 8.78778414431401e-05),
 (43542, 7.170216608239382e-05),
 (43596, 3.099175965346064e-05),
 (44056, 7.693069234809004e-05),
 (43606, 0.00023160063298244133),
 (43607, 8.997657580594096e-06),
 (43569, 0.00010426475255385188),
 (43608, 0.00024763254995525296),
 (43609, 0.0001071280551922446),
 (43615, 2.7557842358

In [ ]:
# convert results from metric functions to dictionary
# def results_to_dictionary(results):
#     results_dict = {}
#     for tupl in results:
#         results_dict.update({tupl[0]:tupl[1]})
#     return(results_dict)
        
        
    

In [ ]:
# TREC setting


In [84]:
import json
# import documents
directory = '/Users/sarab/work/try/TREC/processed_data/wFT/' 
text = {}
for filename in os.listdir(directory):
    with open('C:\\Users\\sarab\\work\\try\\TREC\\processed_data\\wFT\\'+filename, 'r') as file:
        for line in file: 
            dicti = json.loads(line)
            text = {**text, **dicti}
# import questions
topics = {} 
with open('C:\\Users\\sarab\\work\\try\\TREC\\processed_data\\topics', 'r') as file:
        for line in file: 
            dicti = json.loads(line)
            topics = {**topics, **dicti}



In [33]:
# creates pytrec_eval form of results and writes a file
# args: name - name of dictionary, we are creating
#       topics_dict  - dict of question numbers and questions
#       results - metric results we want to convert
def make_dict(topics_dict):
    main_dicti = {}
    for key in topics_dict:
        main_dicti[key] = {}
    return main_dicti

def store_to_dict(this_key, results, main_dicti):
    for tupl in results:
        main_dicti[this_key][tupl[0]] = tupl[1]
    return main_dicti
        
def write_dict(name,main_dicti):  
    file = open(name,"w+") 
    dict_to_write = json.dumps(main_dicti)
    file.write(dict_to_write +"\n")
    file.close() 

    

In [43]:
# testing metrics
multiply0D = make_dict(topics)
multiply1D = make_dict(topics)
multiply02D = make_dict(topics)
multiply3D = make_dict(topics)
sum0D = make_dict(topics)
sum1D = make_dict(topics)
sum2D = make_dict(topics)
sum3D = make_dict(topics)
wsum01D = make_dict(topics)
wsum055D = make_dict(topics)
wsum060D = make_dict(topics)
wsum065D = make_dict(topics)
wsum070D = make_dict(topics)
wsum075D = make_dict(topics)
wsum080D = make_dict(topics)
wsum085D = make_dict(topics)
wsum090D = make_dict(topics)
wsum11D= make_dict(topics)
wsum155D = make_dict(topics)
wsum160D = make_dict(topics)
wsum165D = make_dict(topics)
wsum170D = make_dict(topics)
wsum175D = make_dict(topics)
wsum180D = make_dict(topics)
wsum185D = make_dict(topics)
wsum190D = make_dict(topics)
tfidf_sum0D = make_dict(topics)
tfidf_sum1D = make_dict(topics)
tfidf_sum2D = make_dict(topics)
tfidf_sum3D = make_dict(topics)
tfidf_wsum01D = make_dict(topics)
tfidf_wsum055D = make_dict(topics)
tfidf_wsum060D = make_dict(topics)
tfidf_wsum065D = make_dict(topics)
tfidf_wsum070D = make_dict(topics)
tfidf_wsum075D = make_dict(topics)
tfidf_wsum080D = make_dict(topics)
tfidf_wsum085D = make_dict(topics)
tfidf_wsum090D = make_dict(topics)
tfidf_wsum11D = make_dict(topics)
tfidf_wsum155D = make_dict(topics)
tfidf_wsum160D = make_dict(topics)
tfidf_wsum165D = make_dict(topics)
tfidf_wsum170D = make_dict(topics)
tfidf_wsum175D = make_dict(topics)
tfidf_wsum180D = make_dict(topics)
tfidf_wsum185D = make_dict(topics)
tfidf_wsum190D = make_dict(topics)


for key in topics:
    print(key)
    tokenized = qe.tokenize(topics.get(key), stopWords)
    ext = qe.extend_tokens(tokenized, wv_wiki_en)
    candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=False)
    ext_candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    
    for alpha in [-1,0.55, 0.6,0.65, 0.7, 0.75,0.8,0.85, 0.9]:
        wsum0 = dr.probability_score_sum_weights(tokenized, candidates, text,0, alpha, wv_wiki_en)
        wsum0 = store_to_dict(key, wsum0, 'wsum0'+str(alpha))
        
        wsum1 = dr.probability_score_sum_weights(tokenized+ext, ext_candidates, text,0, alpha, wv_wiki_en)
        wsum1 = to_dict('wsum1'+str(alpha),key, topics, wsum1)
    
    multiply0 = dr.probability_score(tokenized,text, dr.probability_multiply,0)
    multiply0D = store_to_dict(key, multiply0, multiply0D)
    
    multiply1 = dr.probability_score(tokenized+ext,text, dr.probability_multiply,0)
    multiply1 = to_dict('multiply1',key, topics, multiply1)
    
    multiply2 = dr.probability_score(tokenized+ext+ext_candidates,text, dr.probability_multiply,0)
    multiply2 = to_dict('multiply2',key, topics, multiply2)
    
    multiply3 = dr.probability_score(tokenized+candidates,text, dr.probability_multiply,0)
    multiply3 = to_dict('multiply3',key, topics, multiply3)
    
    sum0 = dr.probability_score(tokenized,text, dr.probability_sum,0)
    sum0 = to_dict('sum0',key, topics, sum0)
    
    sum1 = dr.probability_score(tokenized+ext,text, dr.probability_sum,0)
    sum1 = to_dict('sum1',key, topics, sum1)
    
    sum2 = dr.probability_score(tokenized+ext+ext_candidates,text, dr.probability_sum,0)
    sum2 = to_dict('sum2',key, topics, sum2)
    
    sum3 = dr.probability_score(tokenized+candidates,text, dr.probability_sum,0)
    sum3 = to_dict('sum3',key, topics, sum3)
    
    for alpha in [-1,0.55, 0.6,0.65, 0.7, 0.75,0.8,0.85, 0.9]:
        wsum0 = dr.probability_score_sum_weights(tokenized, candidates, text,0, alpha, wv_wiki_en)
        wsum0 = store_to_dict(key, wsum0, wsum0+alpha)
        
        wsum1 = dr.probability_score_sum_weights(tokenized+ext, ext_candidates, text,0, alpha, wv_wiki_en)
        wsum1 = to_dict('wsum1'+str(alpha),key, topics, wsum1)

        
    tfidf_sum0 = dr.tfidf_score(tokenized, text, dr.tfidf_sum,0)
    tfidf_sum0 = to_dict('tfidf_sum0',key, topics, tfidf_sum0)
    
    tfidf_sum1 = dr.tfidf_score(tokenized+ext, text, dr.tfidf_sum,0)
    tfidf_sum1 = to_dict('tfidf_sum1',key, topics, tfidf_sum1)
    
    tfidf_sum2 = dr.tfidf_score(tokenized+ext+ext_candidates, text, dr.tfidf_sum,0)
    tfidf_sum2 = to_dict('tfidf_sum2',key, topics, tfidf_sum2)
    
    tfidf_sum3 = dr.tfidf_score(tokenized+candidates, text, dr.tfidf_sum,0)
    tfidf_sum3 = to_dict('tfidf_sum3',key, topics, tfidf_sum3)
    
    for alpha in [-1,0.55, 0.6,0.65, 0.7, 0.75,0.8,0.85, 0.9]:
        tfidf_wsum0 = dr.tfidf_score(tokenized, text, dr.tfidf_sum_weight,0,candidates,alpha,wv_wiki_en)
        tfidf_wsum0 = to_dict('tfidf_wsum0',key, topics, tfidf_wsum0)
        
        tfidf_wsum1 = dr.tfidf_score(tokenized+ext, text, dr.tfidf_sum_weight,0,ext_candidates,alpha,wv_wiki_en)
        tfidf_wsum1 = to_dict('tfidf_wsum1',key, topics, tfidf_wsum1)
    
write_dict('multiply0',multiply0)

301


TypeError: string indices must be integers

In [85]:
def to_dict(name, this_key,topics_dict,results):
    try:
        
        with open(name, 'r+') as file:
            for line in file: 
                dicti = json.loads(line)
               
            for tupl in results:
                dicti[this_key][tupl[0]] = tupl[1]

            dict_to_write = json.dumps(dicti)
            file.seek(0)
            file.write(dict_to_write +"\n")
            file.truncate()
            file.close() 
           
            
        
        
    except:
        main_dicti = {}
        for key in topics_dict:
            main_dicti[key] = {}
        for tupl in results:
            main_dicti[this_key][tupl[0]] = tupl[1]
    
        file = open(name,"w+") 
        dict_to_write = json.dumps(main_dicti)
        file.write(dict_to_write +"\n")
        file.close() 
      


In [92]:
# testing metrics  

import time

start = time.time()

main_dicti = {}
for key in topics:
    main_dicti[key] = {}

for key in topics:
    print(key)
    tokenized = qe.tokenize(topics.get(key), stopWords)
    ext = qe.extend_tokens(tokenized, wv_wiki_en)
    candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=False)
    ext_candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    
    multiply0 = dr.probability_score(tokenized,text, dr.probability_multiply,0)
    for tupl in multiply0:
        main_dicti[key][tupl[0]] = tupl[1]
        
file = open('multiply0',"w+") 
dict_to_write = json.dumps(main_dicti)
file.write(dict_to_write +"\n")
file.close()

end = time.time()
print(end - start)



301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
1736.8541052341461


In [93]:
import time

start = time.time()

main_dicti1 = {}
for key in topics:
    main_dicti1[key] = {}
    
main_dicti2 = {}
for key in topics:
    main_dicti2[key] = {}
    
main_dicti3 = {}
for key in topics:
    main_dicti3[key] = {}
    
    

for key in topics:
    print(key)
    tokenized = qe.tokenize(topics.get(key), stopWords)
    ext = qe.extend_tokens(tokenized, wv_wiki_en)
    candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=False)
    ext_candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    
        
    multiply1 = dr.probability_score(tokenized+ext,text, dr.probability_multiply,0)
    for tupl in multiply1:
        main_dicti1[key][tupl[0]] = tupl[1]
    
    multiply2 = dr.probability_score(tokenized+ext+ext_candidates,text, dr.probability_multiply,0)
    for tupl in multiply2:
        main_dicti2[key][tupl[0]] = tupl[1]
        
    multiply3 = dr.probability_score(tokenized+candidates,text, dr.probability_multiply,0)
    for tupl in multiply3:
        main_dicti3[key][tupl[0]] = tupl[1]
        
file = open('multiply1',"w+") 
dict_to_write = json.dumps(main_dicti1)
file.write(dict_to_write +"\n")
file.close()

file = open('multiply2',"w+") 
dict_to_write = json.dumps(main_dicti2)
file.write(dict_to_write +"\n")
file.close()

file = open('multiply3',"w+") 
dict_to_write = json.dumps(main_dicti3)
file.write(dict_to_write +"\n")
file.close()

end = time.time()
print(end - start)

301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
5017.071662425995


In [94]:
import time

start = time.time()

main_dicti0 = {}
for key in topics:
    main_dicti0[key] = {}

main_dicti1 = {}
for key in topics:
    main_dicti1[key] = {}
    
main_dicti2 = {}
for key in topics:
    main_dicti2[key] = {}
    
main_dicti3 = {}
for key in topics:
    main_dicti3[key] = {}
    
    

for key in topics:
    print(key)
    tokenized = qe.tokenize(topics.get(key), stopWords)
    ext = qe.extend_tokens(tokenized, wv_wiki_en)
    candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=False)
    ext_candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    

    
    sum0 = dr.probability_score(tokenized,text, dr.probability_sum,0)
    for tupl in sum0:
        main_dicti0[key][tupl[0]] = tupl[1]
    
    sum1 = dr.probability_score(tokenized+ext,text, dr.probability_sum,0)
    for tupl in sum1:
        main_dicti1[key][tupl[0]] = tupl[1]
        
    sum2 = dr.probability_score(tokenized+ext+ext_candidates,text, dr.probability_sum,0)
    for tupl in sum2:
        main_dicti2[key][tupl[0]] = tupl[1]
        
    sum3 = dr.probability_score(tokenized+candidates,text, dr.probability_sum,0)
    for tupl in sum3:
        main_dicti3[key][tupl[0]] = tupl[1]
        
file = open('sum0',"w+") 
dict_to_write = json.dumps(main_dicti0)
file.write(dict_to_write +"\n")
file.close()
        
file = open('sum1',"w+") 
dict_to_write = json.dumps(main_dicti1)
file.write(dict_to_write +"\n")
file.close()

file = open('sum2',"w+") 
dict_to_write = json.dumps(main_dicti2)
file.write(dict_to_write +"\n")
file.close()

file = open('sum3',"w+") 
dict_to_write = json.dumps(main_dicti3)
file.write(dict_to_write +"\n")
file.close()

end = time.time()
print(end - start)

301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
5876.6221425533295


In [95]:
start = time.time()

main_dicti0 = {}
for key in topics:
    main_dicti0[key] = {}

main_dicti1 = {}
for key in topics:
    main_dicti1[key] = {}
    
main_dicti2 = {}
for key in topics:
    main_dicti2[key] = {}
    
main_dicti3 = {}
for key in topics:
    main_dicti3[key] = {}
    
    

for key in topics:
    print(key)
    tokenized = qe.tokenize(topics.get(key), stopWords)
    ext = qe.extend_tokens(tokenized, wv_wiki_en)
    candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=False)
    ext_candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    
    
    tfidf_sum0 = dr.tfidf_score(tokenized, text, dr.tfidf_sum,0)
    for tupl in tfidf_sum0:
        main_dicti0[key][tupl[0]] = tupl[1]
    
    tfidf_sum1 = dr.tfidf_score(tokenized+ext, text, dr.tfidf_sum,0)
    for tupl in tfidf_sum1:
        main_dicti1[key][tupl[0]] = tupl[1]
        
    tfidf_sum2 = dr.tfidf_score(tokenized+ext+ext_candidates, text, dr.tfidf_sum,0)
    for tupl in tfidf_sum2:
        main_dicti2[key][tupl[0]] = tupl[1]
        
    tfidf_sum3 = dr.tfidf_score(tokenized+candidates, text, dr.tfidf_sum,0)
    for tupl in tfidf_sum3:
        main_dicti3[key][tupl[0]] = tupl[1]
        
file = open('tfidf_sum0',"w+") 
dict_to_write = json.dumps(main_dicti0)
file.write(dict_to_write +"\n")
file.close()
        
file = open('tfidf_sum1',"w+") 
dict_to_write = json.dumps(main_dicti1)
file.write(dict_to_write +"\n")
file.close()

file = open('tfidf_sum2',"w+") 
dict_to_write = json.dumps(main_dicti2)
file.write(dict_to_write +"\n")
file.close()

file = open('tfidf_sum3',"w+") 
dict_to_write = json.dumps(main_dicti3)
file.write(dict_to_write +"\n")
file.close()

end = time.time()
print(end - start)

301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
9121.465641260147


In [96]:
start = time.time()

main_dicti0 = {}
for key in topics:
    main_dicti0[key] = {}

main_dicti1 = {}
for key in topics:
    main_dicti1[key] = {}
    
main_dicti2 = {}
for key in topics:
    main_dicti2[key] = {}
    
main_dicti3 = {}
for key in topics:
    main_dicti3[key] = {}
    
main_dicti4 = {}
for key in topics:
    main_dicti4[key] = {}
    
    

for key in topics:
    print(key)
    tokenized = qe.tokenize(topics.get(key), stopWords)
    ext = qe.extend_tokens(tokenized, wv_wiki_en)
    candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=False)
    ext_candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    

    wsum01 = dr.probability_score_sum_weights(tokenized, candidates, text,0, -1, wv_wiki_en)
    for tupl in wsum01:
        main_dicti0[key][tupl[0]] = tupl[1]
        
    wsum006 = dr.probability_score_sum_weights(tokenized, candidates, text,0, 0.6, wv_wiki_en)
    for tupl in wsum006:
        main_dicti1[key][tupl[0]] = tupl[1]
        
    wsum007 = dr.probability_score_sum_weights(tokenized, candidates, text,0, 0.7, wv_wiki_en)
    for tupl in wsum007:
        main_dicti2[key][tupl[0]] = tupl[1]
        
    wsum008 = dr.probability_score_sum_weights(tokenized, candidates, text,0, 0.8, wv_wiki_en)
    for tupl in wsum008:
        main_dicti3[key][tupl[0]] = tupl[1]
        
    wsum009 = dr.probability_score_sum_weights(tokenized, candidates, text,0, 0.9, wv_wiki_en)
    for tupl in wsum009:
        main_dicti4[key][tupl[0]] = tupl[1]
        
        
file = open('wsum01',"w+") 
dict_to_write = json.dumps(main_dicti0)
file.write(dict_to_write +"\n")
file.close()
        
file = open('wsum006',"w+") 
dict_to_write = json.dumps(main_dicti1)
file.write(dict_to_write +"\n")
file.close()

file = open('wsum007',"w+") 
dict_to_write = json.dumps(main_dicti2)
file.write(dict_to_write +"\n")
file.close()

file = open('wsum008',"w+") 
dict_to_write = json.dumps(main_dicti3)
file.write(dict_to_write +"\n")
file.close()

file = open('wsum009',"w+") 
dict_to_write = json.dumps(main_dicti4)
file.write(dict_to_write +"\n")
file.close()

end = time.time()
print(end - start)

301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
16129.118495225906


In [97]:
start = time.time()

main_dicti0 = {}
for key in topics:
    main_dicti0[key] = {}

main_dicti1 = {}
for key in topics:
    main_dicti1[key] = {}
    
main_dicti2 = {}
for key in topics:
    main_dicti2[key] = {}
    
main_dicti3 = {}
for key in topics:
    main_dicti3[key] = {}
    
main_dicti4 = {}
for key in topics:
    main_dicti4[key] = {}
    
    

for key in topics:
    print(key)
    tokenized = qe.tokenize(topics.get(key), stopWords)
    ext = qe.extend_tokens(tokenized, wv_wiki_en)
    candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=False)
    ext_candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    

    wsum11 = dr.probability_score_sum_weights(tokenized+ext, ext_candidates, text,0, -1, wv_wiki_en)
    for tupl in wsum11:
        main_dicti0[key][tupl[0]] = tupl[1]
        
    wsum106 = dr.probability_score_sum_weights(tokenized+ext, ext_candidates, text,0, 0.6, wv_wiki_en)
    for tupl in wsum106:
        main_dicti1[key][tupl[0]] = tupl[1]
        
    wsum107 = dr.probability_score_sum_weights(tokenized+ext, ext_candidates, text,0, 0.7, wv_wiki_en)
    for tupl in wsum107:
        main_dicti2[key][tupl[0]] = tupl[1]
        
    wsum108 = dr.probability_score_sum_weights(tokenized+ext, ext_candidates, text,0, 0.8, wv_wiki_en)
    for tupl in wsum108:
        main_dicti3[key][tupl[0]] = tupl[1]
        
    wsum109 = dr.probability_score_sum_weights(tokenized+ext, ext_candidates, text,0, 0.9, wv_wiki_en)
    for tupl in wsum109:
        main_dicti4[key][tupl[0]] = tupl[1]
        
        
file = open('wsum11',"w+") 
dict_to_write = json.dumps(main_dicti0)
file.write(dict_to_write +"\n")
file.close()
        
file = open('wsum106',"w+") 
dict_to_write = json.dumps(main_dicti1)
file.write(dict_to_write +"\n")
file.close()

file = open('wsum107',"w+") 
dict_to_write = json.dumps(main_dicti2)
file.write(dict_to_write +"\n")
file.close()

file = open('wsum108',"w+") 
dict_to_write = json.dumps(main_dicti3)
file.write(dict_to_write +"\n")
file.close()

file = open('wsum109',"w+") 
dict_to_write = json.dumps(main_dicti4)
file.write(dict_to_write +"\n")
file.close()

end = time.time()
print(end - start)

301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
22787.96092605591


In [98]:
start = time.time()

main_dicti0 = {}
for key in topics:
    main_dicti0[key] = {}

main_dicti1 = {}
for key in topics:
    main_dicti1[key] = {}
    
main_dicti2 = {}
for key in topics:
    main_dicti2[key] = {}
    
main_dicti3 = {}
for key in topics:
    main_dicti3[key] = {}
    
main_dicti4 = {}
for key in topics:
    main_dicti4[key] = {}
    
    

for key in topics:
    print(key)
    tokenized = qe.tokenize(topics.get(key), stopWords)
    ext = qe.extend_tokens(tokenized, wv_wiki_en)
    candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=False)
    ext_candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    

    tfidf_wsum01 = dr.tfidf_score(tokenized, text, dr.tfidf_sum_weight,0,candidates,-1,wv_wiki_en)
    for tupl in tfidf_wsum01:
        main_dicti0[key][tupl[0]] = tupl[1]
        
    tfidf_wsum006 = dr.tfidf_score(tokenized, text, dr.tfidf_sum_weight,0,candidates,0.6,wv_wiki_en)
    for tupl in tfidf_wsum006:
        main_dicti1[key][tupl[0]] = tupl[1]
        
    tfidf_wsum007 = dr.tfidf_score(tokenized, text, dr.tfidf_sum_weight,0,candidates,0.7,wv_wiki_en)
    for tupl in tfidf_wsum007:
        main_dicti2[key][tupl[0]] = tupl[1]
        
    tfidf_wsum008 = dr.tfidf_score(tokenized, text, dr.tfidf_sum_weight,0,candidates,0.8,wv_wiki_en)
    for tupl in tfidf_wsum008:
        main_dicti3[key][tupl[0]] = tupl[1]
        
    tfidf_wsum009 = dr.tfidf_score(tokenized, text, dr.tfidf_sum_weight,0,candidates,0.9,wv_wiki_en)
    for tupl in tfidf_wsum009:
        main_dicti4[key][tupl[0]] = tupl[1]
        
        
file = open('tfidf_wsum01',"w+") 
dict_to_write = json.dumps(main_dicti0)
file.write(dict_to_write +"\n")
file.close()
        
file = open('tfidf_wsum006',"w+") 
dict_to_write = json.dumps(main_dicti1)
file.write(dict_to_write +"\n")
file.close()

file = open('tfidf_wsum007',"w+") 
dict_to_write = json.dumps(main_dicti2)
file.write(dict_to_write +"\n")
file.close()

file = open('tfidf_wsum008',"w+") 
dict_to_write = json.dumps(main_dicti3)
file.write(dict_to_write +"\n")
file.close()

file = open('tfidf_wsum009',"w+") 
dict_to_write = json.dumps(main_dicti4)
file.write(dict_to_write +"\n")
file.close()

end = time.time()
print(end - start)

301
302
303
304
305
306
307


KeyboardInterrupt: 

In [ ]:
start = time.time()

main_dicti0 = {}
for key in topics:
    main_dicti0[key] = {}

main_dicti1 = {}
for key in topics:
    main_dicti1[key] = {}
    
main_dicti2 = {}
for key in topics:
    main_dicti2[key] = {}
    
main_dicti3 = {}
for key in topics:
    main_dicti3[key] = {}
    
main_dicti4 = {}
for key in topics:
    main_dicti4[key] = {}
    
    

for key in topics:
    print(key)
    tokenized = qe.tokenize(topics.get(key), stopWords)
    ext = qe.extend_tokens(tokenized, wv_wiki_en)
    candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=False)
    ext_candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    

    tfidf_wsum11 = dr.tfidf_score(tokenized+ext, text, dr.tfidf_sum_weight,0,ext_candidates,-1,wv_wiki_en)
    for tupl in tfidf_wsum11:
        main_dicti0[key][tupl[0]] = tupl[1]
        
    tfidf_wsum106 = dr.tfidf_score(tokenized+ext, text, dr.tfidf_sum_weight,0,ext_candidates,0.6,wv_wiki_en)
    for tupl in tfidf_wsum106:
        main_dicti1[key][tupl[0]] = tupl[1]
        
    tfidf_wsum107 = dr.tfidf_score(tokenized+ext, text, dr.tfidf_sum_weight,0,ext_candidates,0.7,wv_wiki_en)
    for tupl in tfidf_wsum107:
        main_dicti2[key][tupl[0]] = tupl[1]
        
    tfidf_wsum108 = dr.tfidf_score(tokenized+ext, text, dr.tfidf_sum_weight,0,ext_candidates,0.8,wv_wiki_en)
    for tupl in tfidf_wsum108:
        main_dicti3[key][tupl[0]] = tupl[1]
        
    tfidf_wsum109 = dr.tfidf_score(tokenized+ext, text, dr.tfidf_sum_weight,0,ext_candidates,0.9,wv_wiki_en)
    for tupl in tfidf_wsum109:
        main_dicti4[key][tupl[0]] = tupl[1]
        
        
file = open('tfidf_wsum11',"w+") 
dict_to_write = json.dumps(main_dicti0)
file.write(dict_to_write +"\n")
file.close()
        
file = open('tfidf_wsum106',"w+") 
dict_to_write = json.dumps(main_dicti1)
file.write(dict_to_write +"\n")
file.close()

file = open('tfidf_wsum107',"w+") 
dict_to_write = json.dumps(main_dicti2)
file.write(dict_to_write +"\n")
file.close()

file = open('tfidf_wsum108',"w+") 
dict_to_write = json.dumps(main_dicti3)
file.write(dict_to_write +"\n")
file.close()

file = open('tfidf_wsum109',"w+") 
dict_to_write = json.dumps(main_dicti4)
file.write(dict_to_write +"\n")
file.close()

end = time.time()
print(end - start)

In [89]:
# testing metrics  

import time

start = time.time()
for key in ['301','302','303','304','305']:
    print(key)
    tokenized = qe.tokenize(topics.get(key), stopWords)
    ext = qe.extend_tokens(tokenized, wv_wiki_en)
    candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=False)
    ext_candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    
    multiply0 = dr.probability_score(tokenized,text, dr.probability_multiply,0)
    multiply0 = to_dict('multiply0',key, topics, multiply0)

end = time.time()
print(end - start)

301
302
303
304
305
59.007657527923584


In [82]:
    
    multiply1 = dr.probability_score(tokenized+ext,text, dr.probability_multiply,0)
    multiply1 = to_dict('multiply1',key, topics, multiply1)
    
    multiply2 = dr.probability_score(tokenized+ext+ext_candidates,text, dr.probability_multiply,0)
    multiply2 = to_dict('multiply2',key, topics, multiply2)
    
    multiply3 = dr.probability_score(tokenized+candidates,text, dr.probability_multiply,0)
    multiply3 = to_dict('multiply3',key, topics, multiply3)
    
    sum0 = dr.probability_score(tokenized,text, dr.probability_sum,0)
    sum0 = to_dict('sum0',key, topics, sum0)
    
    sum1 = dr.probability_score(tokenized+ext,text, dr.probability_sum,0)
    sum1 = to_dict('sum1',key, topics, sum1)
    
    sum2 = dr.probability_score(tokenized+ext+ext_candidates,text, dr.probability_sum,0)
    sum2 = to_dict('sum2',key, topics, sum2)
    
    sum3 = dr.probability_score(tokenized+candidates,text, dr.probability_sum,0)
    sum3 = to_dict('sum3',key, topics, sum3)
    
    for alpha in [-1, 0.6, 0.7, 0.8, 0.9]:
        wsum0 = dr.probability_score_sum_weights(tokenized, candidates, text,0, alpha, wv_wiki_en)
        wsum0 = to_dict('wsum0'+str(alpha),key, topics, wsum0)
        
        wsum1 = dr.probability_score_sum_weights(tokenized+ext, ext_candidates, text,0, alpha, wv_wiki_en)
        wsum1 = to_dict('wsum1'+str(alpha),key, topics, wsum1)
        
    tfidf_sum0 = dr.tfidf_score(tokenized, text, dr.tfidf_sum,0)
    tfidf_sum0 = to_dict('tfidf_sum0',key, topics, tfidf_sum0)
    
    tfidf_sum1 = dr.tfidf_score(tokenized+ext, text, dr.tfidf_sum,0)
    tfidf_sum1 = to_dict('tfidf_sum1',key, topics, tfidf_sum1)
    
    tfidf_sum2 = dr.tfidf_score(tokenized+ext+ext_candidates, text, dr.tfidf_sum,0)
    tfidf_sum2 = to_dict('tfidf_sum2',key, topics, tfidf_sum2)
    
    tfidf_sum3 = dr.tfidf_score(tokenized+candidates, text, dr.tfidf_sum,0)
    tfidf_sum3 = to_dict('tfidf_sum3',key, topics, tfidf_sum3)
    
    for alpha in [-1, 0.6, 0.7, 0.8, 0.9]:
        tfidf_wsum0 = dr.tfidf_score(tokenized, text, dr.tfidf_sum_weight,0,candidates,alpha,wv_wiki_en)
        tfidf_wsum0 = to_dict('tfidf_wsum0'+str(alpha),key, topics, tfidf_wsum0)
        
        tfidf_wsum1 = dr.tfidf_score(tokenized+ext, text, dr.tfidf_sum_weight,0,ext_candidates,alpha,wv_wiki_en)
        tfidf_wsum1 = to_dict('tfidf_wsum1'+str(alpha),key, topics, tfidf_wsum1)
   
        

    
    
    
    

301


KeyboardInterrupt: 

In [ ]:
topics_results

In [ ]:
key = '305'
tokenized = qe.tokenize(topics.get(key), stopWords)
ext = qe.extend_tokens(tokenized, wv_wiki_en)
candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    
multiply0 = dr.probability_score(tokenized,text, dr.probability_sum,10)

In [ ]:
print(len(multiply0))
print(multiply0[0])

In [56]:
multiply0 = to_dict('multiply0',key, topics, multiply0)

210158


TypeError: unhashable type: 'slice'

In [54]:
multiply0.get('305')

{'F': 'T'}

In [14]:
# za mac
import json
def import_file(file_path, file_name):
    file_name = {} 
    with open(file_path, 'r') as file:
            for line in file: 
                dicti = json.loads(line)
                file_name = {**file_name, **dicti}

            
            
def evaluate(file_path,file_name, qrel):
    import_file(file_path, file_name)
    evaluator = pytrec_eval.RelevanceEvaluator(qrel, {'map', 'ndcg'})
    file = open("written_"+str(file_name),"w+") 
    dict_to_write = json.dumps(evaluator.evaluate(run), indent=1)
    file.write(dict_to_write +"\n")
    file.close()

            

In [ ]:
# functions that convert data into pytrec_eval form

In [311]:
#documents (TREC disk 4 FT)
import os
import json
import xml.etree.ElementTree as ElementTree
dir_name = "FT924"
directory = '/Users/sarab/work/try/TREC/TREC-Disk-4/FT/' + dir_name
directory_dict = {}
for filename in os.listdir(directory):
    with open('C:\\Users\\sarab\\work\\try\\TREC\\TREC-Disk-4\\FT\\'+dir_name+'\\'+filename, 'r') as f:   # Reading file
        xml = f.read()
 
    xml = '<ROOT>' + xml + '</ROOT>'   # Let's add a root tag

    root = ElementTree.fromstring(xml)

    # Simple loop through each document
    docs_json = {}
    count = 1
    for doc in root:
       # print(doc)
        #doc_json = {'DOCNO': doc.find('DOCNO').text.strip(), 'TEXT': doc.find('TEXT').text.strip()}
        try:
            doc_json = {doc.find('DOCNO').text.strip(): doc.find('TEXT').text.strip()}
        except:
            print(doc.find('DOCNO').text.strip(), count)
            count = count +1
            doc_json = {doc.find('DOCNO').text.strip(): 'empty'}
        docs_json.update(doc_json)
    directory_dict = {**directory_dict, **docs_json}
    continue
    
file = open("written_"+dir_name,"w+") 
dict_to_write = json.dumps(directory_dict)
file.write(dict_to_write +"\n")
file.close()
    

FT924-11838 1


In [312]:
#documents (TREC disk 4 FT)
import os
import json
import xml.etree.ElementTree as ElementTree
dir_name = "01"
directory = '/Users/sarab/work/try/TREC/TREC-Disk-4/FR94/' + dir_name
directory_dict = {}
for filename in os.listdir(directory):
    with open('C:\\Users\\sarab\\work\\try\\TREC\\TREC-Disk-4\\FR94\\'+dir_name+'\\'+filename, 'r') as f:   # Reading file
        xml = f.read()
 
    xml = '<ROOT>' + xml + '</ROOT>'   # Let's add a root tag

    root = ElementTree.fromstring(xml)

    # Simple loop through each document
    docs_json = {}
    count = 1
    for doc in root:
       # print(doc)
        #doc_json = {'DOCNO': doc.find('DOCNO').text.strip(), 'TEXT': doc.find('TEXT').text.strip()}
        try:
            doc_json = {doc.find('DOCNO').text.strip(): doc.find('TEXT').text.strip()}
        except:
            print(doc.find('DOCNO').text.strip(), count)
            count = count +1
            doc_json = {doc.find('DOCNO').text.strip(): 'empty'}
        docs_json.update(doc_json)
    directory_dict = {**directory_dict, **docs_json}
    continue
    
file = open("written_"+dir_name,"w+") 
dict_to_write = json.dumps(directory_dict)
file.write(dict_to_write +"\n")
file.close()

ParseError: undefined entity: line 19, column 0 (<string>)

In [146]:
#documents (TREC disk 4 and 5)
import os
import json
import xml.etree.ElementTree as ElementTree
dir_name = "F"
directory = '/Users/sarab/work/try/TREC/TREC-Disk-5/FBIS/' + dir_name
directory_dict = {}
for filename in os.listdir(directory):
    print(filename)
    with open('C:\\Users\\sarab\\work\\try\\TREC\\TREC-Disk-5\\FBIS\\'+dir_name+'\\'+filename, 'r') as f:   # Reading file
        xml = f.read()
    parser = ElementTree.XMLParser(encoding="utf-8")
    
    xml = '<ROOT>' + xml + '</ROOT>'   # Let's add a root tag
    
    root = ElementTree.fromstring(xml, parser = parser)

    # Simple loop through each document
    docs_json = {}
    for doc in root:
        #doc_json = {'DOCNO': doc.find('DOCNO').text.strip(), 'TEXT': doc.find('TEXT').text.strip()}
        doc_json = {doc.find('DOCNO').text.strip(): doc.text.strip()}
        docs_json.update(doc_json)
    directory_dict = {**directory_dict, **docs_json}
    continue
    
file = open("written_"+dir_name,"w+") 
dict_to_write = json.dumps(directory_dict)
file.write(dict_to_write +"\n")
file.close()
    

FB396001
FB396002
FB396003


ParseError: not well-formed (invalid token): line 14, column 5 (<string>)

In [286]:
#documents (TREC disk 4 and 5)
# import os
# import json
# import xml.etree.ElementTree as ElementTree
# dir_name = "L"
# directory = '/Users/sarab/work/try/TREC/TREC-Disk-5/LATIMES/' + dir_name
# directory_dict = {}
# for filename in os.listdir(directory):
#     print(filename)
#     with open('C:\\Users\\sarab\\work\\try\\TREC\\TREC-Disk-5\\LATIMES\\'+dir_name+'\\'+filename, 'r') as f:   # Reading file
#         xml = f.read()
   
    
#     xml = '<ROOT>' + xml + '</ROOT>'   # Let's add a root tag
    
#     root = ElementTree.fromstring(xml)

#     # Simple loop through each document
#     docs_json = {}
#     for doc in root:
#         print(doc.find('TEXT').text)
#         #doc_json = {'DOCNO': doc.find('DOCNO').text.strip(), 'TEXT': doc.find('TEXT').text.strip()}
#         doc_json = {doc.find('DOCNO').text.strip(): doc.find('TEXT').text.strip()}
#         docs_json.update(doc_json)
#     directory_dict = {**directory_dict, **docs_json}
#     continue
    
# file = open("written_"+dir_name,"w+") 
# dict_to_write = json.dumps(directory_dict)
# file.write(dict_to_write +"\n")
# file.close()

LA010189




























































































































































AttributeError: 'NoneType' object has no attribute 'text'

In [321]:
import json
import xml.etree.ElementTree as ElementTree
dir_name = "L"
directory = '/Users/sarab/work/try/TREC/TREC-Disk-5/LATIMES/' + dir_name
directory_dict = {}
count = 1
for filename in os.listdir(directory):
    with open('C:\\Users\\sarab\\work\\try\\TREC\\TREC-Disk-5\\LATIMES\\'+dir_name+'\\'+filename, 'r') as f:   # Reading file
        xml = f.read()
 
    xml = '<ROOT>' + xml + '</ROOT>'   # Let's add a root tag

    root = ElementTree.fromstring(xml)

    # Simple loop through each document
    docs_json = {}
    
    for doc in root:
       # print(doc)
        #doc_json = {'DOCNO': doc.find('DOCNO').text.strip(), 'TEXT': doc.find('TEXT').text.strip()}
        try:
            doc_json = {doc.find('DOCNO').text.strip(): doc.find('TEXT').text.strip()}
        except:
            print(doc.find('DOCNO').text.strip(), count)
            count = count +1
            doc_json = {doc.find('DOCNO').text.strip(): 'empty'}
        docs_json.update(doc_json)
    directory_dict = {**directory_dict, **docs_json}
    continue
    
file = open("written_"+dir_name,"w+") 
dict_to_write = json.dumps(directory_dict)
file.write(dict_to_write +"\n")
file.close()

LA010189-0078 1
LA010189-0142 2
LA010189-0152 3
LA010189-0165 4
LA010189-0180 5
LA010290-0001 6
LA010290-0002 7
LA010290-0003 8
LA010290-0004 9
LA010290-0060 10
LA010290-0114 11
LA010290-0122 12
LA010290-0124 13
LA010290-0125 14
LA010389-0001 15
LA010389-0002 16
LA010389-0044 17
LA010389-0046 18
LA010389-0050 19
LA010389-0065 20
LA010389-0080 21
LA010389-0096 22
LA010390-0083 23
LA010390-0103 24
LA010390-0152 25
LA010390-0156 26
LA010489-0018 27
LA010489-0021 28
LA010489-0042 29
LA010489-0047 30
LA010490-0001 31
LA010490-0002 32
LA010490-0008 33
LA010490-0021 34
LA010490-0022 35
LA010490-0023 36
LA010490-0026 37
LA010490-0067 38
LA010490-0088 39
LA010490-0148 40
LA010490-0207 41
LA010490-0228 42
LA010589-0004 43
LA010589-0009 44
LA010589-0011 45
LA010589-0012 46
LA010589-0034 47
LA010589-0080 48
LA010589-0091 49
LA010589-0200 50
LA010589-0207 51
LA010589-0213 52
LA010590-0001 53
LA010590-0010 54
LA010590-0048 55
LA010590-0076 56
LA010590-0113 57
LA010590-0141 58
LA010590-0181 59
LA0105

LA021690-0001 507
LA021690-0074 508
LA021690-0095 509
LA021690-0166 510
LA021690-0173 511
LA021690-0182 512
LA021690-0184 513
LA021690-0186 514
LA021690-0187 515
LA021789-0069 516
LA021789-0153 517
LA021789-0167 518
LA021789-0179 519
LA021790-0003 520
LA021790-0004 521
LA021889-0002 522
LA021889-0017 523
LA021889-0056 524
LA021889-0078 525
LA021889-0119 526
LA021889-0120 527
LA021889-0121 528
LA021889-0132 529
LA021890-0074 530
LA021890-0146 531
LA021989-0053 532
LA021989-0119 533
LA021989-0144 534
LA021989-0168 535
LA021990-0012 536
LA021990-0057 537
LA022089-0059 538
LA022090-0002 539
LA022090-0003 540
LA022090-0132 541
LA022090-0137 542
LA022090-0141 543
LA022090-0144 544
LA022189-0002 545
LA022189-0103 546
LA022189-0130 547
LA022190-0011 548
LA022190-0055 549
LA022190-0065 550
LA022190-0166 551
LA022190-0171 552
LA022190-0174 553
LA022190-0178 554
LA022190-0179 555
LA022190-0180 556
LA022289-0002 557
LA022289-0051 558
LA022289-0097 559
LA022289-0098 560
LA022289-0146 561
LA022289-0

LA032989-0005 965
LA032989-0008 966
LA032989-0138 967
LA032990-0001 968
LA032990-0002 969
LA032990-0003 970
LA032990-0005 971
LA032990-0006 972
LA032990-0007 973
LA032990-0011 974
LA032990-0144 975
LA032990-0147 976
LA032990-0238 977
LA032990-0239 978
LA032990-0240 979
LA032990-0241 980
LA033089-0052 981
LA033089-0059 982
LA033089-0063 983
LA033089-0090 984
LA033089-0095 985
LA033089-0103 986
LA033089-0125 987
LA033089-0128 988
LA033089-0161 989
LA033089-0163 990
LA033089-0181 991
LA033090-0002 992
LA033090-0021 993
LA033090-0054 994
LA033090-0065 995
LA033090-0187 996
LA033090-0191 997
LA033090-0192 998
LA033090-0193 999
LA033090-0200 1000
LA033090-0201 1001
LA033090-0202 1002
LA033189-0007 1003
LA033189-0014 1004
LA033189-0015 1005
LA033189-0019 1006
LA033189-0034 1007
LA033189-0098 1008
LA033189-0180 1009
LA033190-0004 1010
LA033190-0009 1011
LA033190-0057 1012
LA033190-0117 1013
LA040189-0052 1014
LA040189-0077 1015
LA040189-0122 1016
LA040189-0130 1017
LA040189-0139 1018
LA040190-

LA050389-0002 1414
LA050389-0069 1415
LA050389-0136 1416
LA050389-0153 1417
LA050390-0021 1418
LA050390-0052 1419
LA050390-0072 1420
LA050390-0084 1421
LA050390-0101 1422
LA050390-0105 1423
LA050489-0005 1424
LA050489-0006 1425
LA050489-0048 1426
LA050489-0072 1427
LA050489-0082 1428
LA050489-0150 1429
LA050489-0161 1430
LA050489-0188 1431
LA050490-0001 1432
LA050490-0003 1433
LA050490-0031 1434
LA050490-0081 1435
LA050490-0152 1436
LA050490-0170 1437
LA050490-0191 1438
LA050589-0006 1439
LA050589-0007 1440
LA050589-0015 1441
LA050589-0049 1442
LA050589-0053 1443
LA050590-0069 1444
LA050689-0001 1445
LA050689-0002 1446
LA050689-0006 1447
LA050689-0007 1448
LA050689-0009 1449
LA050690-0014 1450
LA050690-0030 1451
LA050690-0077 1452
LA050690-0177 1453
LA050690-0178 1454
LA050789-0058 1455
LA050789-0060 1456
LA050789-0084 1457
LA050789-0108 1458
LA050789-0164 1459
LA050790-0019 1460
LA050790-0107 1461
LA050790-0109 1462
LA050790-0110 1463
LA050790-0126 1464
LA050889-0002 1465
LA050889-010

LA060989-0001 1856
LA060989-0002 1857
LA060989-0033 1858
LA060989-0034 1859
LA060989-0035 1860
LA060989-0037 1861
LA060989-0076 1862
LA060989-0130 1863
LA060990-0040 1864
LA061089-0009 1865
LA061089-0024 1866
LA061089-0110 1867
LA061090-0052 1868
LA061090-0082 1869
LA061090-0125 1870
LA061090-0129 1871
LA061090-0170 1872
LA061090-0217 1873
LA061090-0231 1874
LA061189-0096 1875
LA061189-0108 1876
LA061189-0113 1877
LA061189-0114 1878
LA061189-0116 1879
LA061189-0170 1880
LA061189-0174 1881
LA061189-0176 1882
LA061190-0033 1883
LA061190-0118 1884
LA061190-0126 1885
LA061289-0001 1886
LA061289-0113 1887
LA061290-0001 1888
LA061290-0002 1889
LA061290-0003 1890
LA061290-0044 1891
LA061290-0046 1892
LA061290-0115 1893
LA061290-0171 1894
LA061389-0102 1895
LA061389-0125 1896
LA061390-0024 1897
LA061390-0027 1898
LA061390-0033 1899
LA061390-0106 1900
LA061390-0127 1901
LA061390-0165 1902
LA061390-0166 1903
LA061390-0168 1904
LA061489-0005 1905
LA061489-0031 1906
LA061489-0046 1907
LA061490-001

LA072089-0015 2314
LA072089-0108 2315
LA072089-0147 2316
LA072089-0171 2317
LA072089-0212 2318
LA072090-0155 2319
LA072090-0168 2320
LA072090-0172 2321
LA072090-0178 2322
LA072090-0179 2323
LA072090-0180 2324
LA072189-0001 2325
LA072189-0002 2326
LA072189-0048 2327
LA072189-0052 2328
LA072190-0006 2329
LA072190-0007 2330
LA072190-0008 2331
LA072190-0009 2332
LA072190-0105 2333
LA072190-0107 2334
LA072190-0114 2335
LA072289-0004 2336
LA072289-0017 2337
LA072289-0022 2338
LA072289-0112 2339
LA072290-0052 2340
LA072290-0055 2341
LA072290-0057 2342
LA072290-0060 2343
LA072290-0153 2344
LA072290-0163 2345
LA072290-0195 2346
LA072389-0072 2347
LA072389-0114 2348
LA072389-0137 2349
LA072389-0192 2350
LA072390-0015 2351
LA072390-0017 2352
LA072390-0018 2353
LA072390-0055 2354
LA072390-0123 2355
LA072390-0135 2356
LA072489-0045 2357
LA072489-0108 2358
LA072490-0003 2359
LA072490-0100 2360
LA072490-0181 2361
LA072589-0008 2362
LA072589-0043 2363
LA072589-0053 2364
LA072589-0054 2365
LA072590-000

LA082990-0026 2764
LA083089-0003 2765
LA083089-0037 2766
LA083089-0138 2767
LA083090-0001 2768
LA083090-0015 2769
LA083090-0115 2770
LA083090-0155 2771
LA083090-0183 2772
LA083090-0245 2773
LA083090-0252 2774
LA083090-0256 2775
LA083090-0263 2776
LA083189-0027 2777
LA083189-0032 2778
LA083189-0037 2779
LA083189-0038 2780
LA083189-0099 2781
LA083189-0119 2782
LA083190-0034 2783
LA083190-0035 2784
LA083190-0063 2785
LA083190-0154 2786
LA083190-0166 2787
LA083190-0172 2788
LA090189-0001 2789
LA090189-0003 2790
LA090189-0073 2791
LA090189-0095 2792
LA090189-0147 2793
LA090189-0166 2794
LA090190-0022 2795
LA090190-0056 2796
LA090190-0121 2797
LA090289-0001 2798
LA090289-0041 2799
LA090289-0116 2800
LA090290-0021 2801
LA090290-0028 2802
LA090290-0050 2803
LA090290-0079 2804
LA090290-0155 2805
LA090389-0066 2806
LA090390-0009 2807
LA090390-0010 2808
LA090489-0002 2809
LA090489-0014 2810
LA090490-0095 2811
LA090490-0142 2812
LA090490-0146 2813
LA090589-0001 2814
LA090589-0090 2815
LA090589-011

LA101689-0018 3242
LA101689-0019 3243
LA101689-0020 3244
LA101689-0024 3245
LA101689-0042 3246
LA101689-0079 3247
LA101689-0100 3248
LA101690-0004 3249
LA101690-0013 3250
LA101690-0164 3251
LA101690-0166 3252
LA101690-0176 3253
LA101690-0180 3254
LA101789-0001 3255
LA101789-0004 3256
LA101789-0010 3257
LA101789-0012 3258
LA101789-0078 3259
LA101789-0154 3260
LA101789-0165 3261
LA101790-0054 3262
LA101790-0064 3263
LA101790-0067 3264
LA101790-0082 3265
LA101790-0147 3266
LA101790-0151 3267
LA101790-0164 3268
LA101889-0001 3269
LA101889-0002 3270
LA101889-0003 3271
LA101889-0004 3272
LA101889-0039 3273
LA101889-0179 3274
LA101889-0180 3275
LA101889-0181 3276
LA101889-0182 3277
LA101889-0183 3278
LA101889-0184 3279
LA101889-0185 3280
LA101889-0186 3281
LA101890-0001 3282
LA101890-0003 3283
LA101890-0025 3284
LA101890-0026 3285
LA101890-0041 3286
LA101890-0042 3287
LA101890-0043 3288
LA101890-0137 3289
LA101890-0218 3290
LA101890-0220 3291
LA101890-0246 3292
LA101890-0269 3293
LA101890-027

LA112089-0046 3687
LA112089-0093 3688
LA112089-0102 3689
LA112089-0115 3690
LA112089-0130 3691
LA112089-0135 3692
LA112089-0136 3693
LA112089-0137 3694
LA112090-0001 3695
LA112090-0042 3696
LA112090-0054 3697
LA112090-0071 3698
LA112090-0189 3699
LA112189-0051 3700
LA112189-0055 3701
LA112189-0171 3702
LA112189-0176 3703
LA112189-0177 3704
LA112190-0153 3705
LA112190-0166 3706
LA112289-0005 3707
LA112289-0006 3708
LA112289-0070 3709
LA112289-0160 3710
LA112289-0166 3711
LA112290-0036 3712
LA112290-0083 3713
LA112290-0123 3714
LA112290-0134 3715
LA112290-0188 3716
LA112389-0009 3717
LA112389-0027 3718
LA112389-0082 3719
LA112389-0097 3720
LA112390-0035 3721
LA112390-0103 3722
LA112390-0106 3723
LA112390-0168 3724
LA112489-0035 3725
LA112489-0129 3726
LA112489-0140 3727
LA112489-0155 3728
LA112490-0001 3729
LA112490-0002 3730
LA112490-0053 3731
LA112589-0001 3732
LA112589-0002 3733
LA112589-0050 3734
LA112590-0065 3735
LA112590-0069 3736
LA112590-0092 3737
LA112590-0103 3738
LA112590-011

LA123090-0022 4135
LA123090-0032 4136
LA123090-0040 4137
LA123090-0062 4138
LA123090-0096 4139
LA123090-0110 4140
LA123189-0079 4141
LA123189-0099 4142
LA123189-0100 4143
LA123189-0101 4144
LA123189-0106 4145
LA123189-0107 4146
LA123190-0020 4147
LA123190-0028 4148
LA123190-0046 4149
LA123190-0096 4150
LA123190-0107 4151
LA123190-0111 4152
LA123190-0124 4153
LA123190-0128 4154


In [287]:
len(directory_dict)

0

In [244]:
# questions (300 - 450 ad hoc)
import re
directory = '/Users/sarab/work/try/TREC/topics/' 
#directory_list = []
question_dictionary = {}
for filename in os.listdir(directory):

    with open('C:\\Users\\sarab\\work\\try\\TREC\\topics\\'+filename) as f:
        file = f.read()
        numbers = re.findall('<num>\sNumber:\s\d+',file,re.MULTILINE)
        questions = re.findall('<desc>\sDescription:(.*?)<narr>',file,re.DOTALL)
        for i in range(len(numbers)):
            number = re.search('\d+', numbers[i])
            number = number.group(0)
            question = questions[i].replace('\n','')
            question_dictionary.update({number: question})

file = open("topics","w+") 
dict_to_write = json.dumps(question_dictionary)
file.write(dict_to_write)
file.close()

In [243]:
# qrels for questions 300 - 450

import re
directory = '/Users/sarab/work/try/TREC/qrels/' 
tuple_list= []
for filename in os.listdir(directory):
    with open('C:\\Users\\sarab\\work\\try\\TREC\\qrels\\'+filename) as f:
        file = f.read()
       # questions = re.findall(r'(\d+)\s0\s.+\s\d\n',file)
        documents = re.findall(r'\d+\s0\s(.+)\s\d\n',file)
        relavances = re.findall(r'(\d+)\s0\s.+\s(\d)\n',file)
        if len(relavances) == len(documents) :
            for i in range(len(relavances)):
                tuple_list.append((relavances[i][0], documents[i], relavances[i][1]))
                
        else:
            print('Error')
            
#write in form for pytrec_eval:
main_dict = {}
for nb in range(301,451):
    main_dict[nb] = {}
for tupl in tuple_list:
    main_dict[int(float(tupl[0]))][tupl[1]] = tupl[2]

file = open("qrels","w+") 
dict_to_write = json.dumps(main_dict)
file.write(dict_to_write)
file.close()
    


            


In [108]:
class Decoder(json.JSONDecoder):
    def decode(self, s):
        result = super().decode(s)  # result = super(Decoder, self).decode(s) for Python 2.x
        return self._decode(result)

    def _decode(self, o):
        if isinstance(o, str):
            try:
                return int(o)
            except ValueError:
                return o
        elif isinstance(o, dict):
            return {k: self._decode(v) for k, v in o.items()}
        elif isinstance(o, list):
            return [self._decode(v) for v in o]
        else:
            return o

In [109]:
def import_file(file_path):
    file_dict = {}
    with open(file_path, 'r') as file:
        for line in file:
            dicti = json.loads(line, cls=Decoder)
            file_dict = {**file_dict, **dicti}
    return file_dict


In [110]:
qrel = import_file("/Users/sarab/work/try/TREC/processed_data/qrels")

In [111]:
qrel.get("301").get('CR93E-10279')


0

In [ ]:
# analysing TREC results

In [140]:
import ast
directory = "/Users/sarab/work/try/TREC/processed_data/eval_results"
map_results = {}

for filename in os.listdir(directory):
    map_results[filename] = {}
    q301_350 = 0
    q351_400 = 0
    q401_450 = 0
    map_val = 0
    count = 0
    c1 = 0
    c2 = 0
    c3 = 0

    with open(directory + '/' + filename, 'r') as file:
        results = json.loads(file.read(), cls=Decoder)
    for key,value in results.items():
        key = int(key)
        val = value.get("map")
        if 301 <= key <=350:
            c1 += 1
            q301_350 = q301_350 + val
                
        elif 351 <= key <=400:
            c2 += 1
            q351_400 = q351_400 + val
            
        elif 401 <= key <=450:
            c3 += 1
            q401_450 = q401_450 + val
            
        else:
            print('error')
            
        map_val = map_val + val
        count += 1
    map_final = map_val / count
    q301_350 = q301_350 / c1
    q351_400 = q351_400 / c2
    q401_450 = q401_450 / c3
    map_results[filename]['q301_450'] = map_final
    map_results[filename]['q301_350'] = q301_350
    map_results[filename]['q351_400'] = q401_450
    map_results[filename]['q401_450'] = q401_450
    if c1 !=50 or c2 !=50 or c1 !=50 or count!=150 :
        print('error_c')
    
    


In [143]:
import pandas
df = pandas.DataFrame.from_dict(map_results,'index')
df.sort_values(by=['q301_450'], ascending=False)

,q301_450,q301_350,q351_400,q401_450
eval_tfidf_sum0,0.010130,0.016936,0.007630,0.007630
eval_tfidf_sum2,0.009466,0.013976,0.007043,0.007043
eval_tfidf_sum3,0.009060,0.014555,0.006632,0.006632
eval_tfidf_sum1,0.009025,0.014215,0.006682,0.006682
eval_wsum109,0.003533,0.005219,0.002864,0.002864
eval_wsum108,0.003533,0.005219,0.002864,0.002864
eval_wsum107,0.003533,0.005219,0.002864,0.002864
eval_wsum106,0.003533,0.005219,0.002864,0.002864
eval_wsum11,0.003533,0.005219,0.002864,0.002864
eval_sum2,0.003533,0.005219,0.002864,0.002864
